###### Coursera  | Andrew Ng's Deep Learning Class | Course2. Improving Deep Neural Networks_Hyperparameter tuning, Regularization and Optimization | Week3. ,,

# [1]. Hyperparameter tuning

### [1-1]. Tuning process

신경망 모형을 학습시키려면 결정해야 하는 하이퍼-파라미터가 많다. 이런 값들을 어떻게 잘 정할 수 있을지 알아보자. 

대표적인 하이퍼-파라미터들은 아래와 같다.

- $\alpha$ - learning rate


- $\beta$ - 모멘텀 방법의 가중치
- $\beta_1$, $\beta_2$, $\epsilon$ - Adam의 파라미터들


- $L$ - 신경망의 히든 레이어 갯수
- $l_n$ - 각 히든 레이어의 유닛 수


- $\text{learning rate decay}$
- $\text{mini-batch size}$

이들 하이퍼-파라미터 중 가장 중요하다고 생각되는 것은 learning rate $\alpha$이고, 다음으로 중요한 것들은 $\beta$, $L$, $\text{mini-batch size}$이다. 참고로 $\beta$값의 초기값으로는 주로 0.9를 사용한다. 반면 Adam의 파라미터는 거의 튜닝하지 않는데, 그냥 $\beta_1$값으로 0.9, $\beta_2$값으로는 0.999, $\epsilon$값으로는 $10^{-8}$을 사용한다.

물론 이런 중요도는 전문가에 따라서 혹은 상황에 따라 달라질 수 있다.


그렇다면 이런 값들을 어떻게 튜닝해야 할까?

초기 머신러닝 분야에서는 각 파라미터를 일정한 간격의 그리드로 나누고 모든 경우에 대해 모형을 테스트 하고 가장 좋은 값을 고르는 방법을 사용했었다. 이런 방법은 결정해야하는 하이퍼-파라미터의 수가 많지 않을 때에는 잘 동작한다. 하지만 하이퍼-파라미터수가 많아지면 모든 경우를 다 시도해보는 것은 어려워진다.

반면 딥러닝에서는 랜덤하게 생성된 하이퍼-파라미터값을 고르고 그 값에 대한 성능을 테스트 하는데, 이렇게 하는 이유는 여러 값들을 테스트해보기 전에는 사실 어떤 하이퍼-파라미터가 중요한지(모형 성능에 큰 영향을 주는지) 확신하기 어렵워서 이다. 

예를들어 아래 슬라이드에서 하이퍼-파라미터1이 $\alpha$이고 하이퍼-파라미터2가 $\epsilon$이라면 사실 하이퍼-파라미터2를 바꾸는 것은 큰 의미가 없을 것이고(Adam에서 $\epsilon$은 모형 성능에 큰 영향이 없으므로), 하이퍼-파라미터1만이 성능에 주요한 영향을 줄 것이다. 따라서 아래 슬라이드 우측과 같은 grid search를 할 경우 25개의 하이퍼-파라미터 쌍에 대해 테스트를 하지만 결국 $\alpha$는 5개 다른 값만을 테스트 하게 된다. 반면 슬라이드 우측과 같이 랜덤한 값을 사용한 경우 25개에 가까운 서로 다른 $\alpha$값을 테스트 해볼 수 있게 된다. 이런 점은 하이퍼-파라미터수가 많아질 수록 그 효과가 커지게 된다.


이렇게 하이퍼-파라미터를 샘플링하는 방법을 사용할 때 유용한 접근법 중 하나가 바로 **거친 값으로 시작하여 점차 고운값을 사용하라.**(coarse to fine)이다. 우선 큰 간격으로 넓은 범위의 값에서 샘플링한 점들에 대해 테스트를 진행하고, 이들 중 괜찮은 하이퍼-파라미터 조합이 발견되면, 그 값을 중심으로 일정 범위에서 값을 샘플링하여 테스트 하는 것이다.(점점 적절한 곳을 zoom-in하는 것과 같다.)

결론적으로  하이퍼-파라미터 튜닝 시 grid search보다 랜덤 샘플링을 이용하는 것이 효율적이다.

### [1-2]. Using an appropriate scale to pick hyperparameters

하이퍼-파라미터 튜닝시에 랜덤 샘플링한 값을 이용하면 효율적이라고 했었다. 그런데 이때의 랜덤 샘플링이 단순히 어떤 범위의 uniform 분포에서 추출하는 것을 의미하지는 않는다. 이는 하이퍼-파라미터에 따라 특정 범위의 값을 좀 더 조밀하게 혹은 듬성듬성 테스트 해야할 필요가 있기 때문이다. 

예를들어 어떤 히든 유닛의 수를 결정할 때 대략 50개에서 100개 사이의 값이 적절할 것이라면 Uniform(50, 100)에서의 샘플링 값을 테스트 하는 것이 그럴듯 하다. (혹은 히든 레이어의 갯수 또한 마찬가지이다.) 

그런데 learning rate $\alpha$와 같은 하이퍼-파라미터는 그 성격이 다르다. 예를들어 0.0001에서 1 사이의 $\alpha$ 값을 테스트해야 한다고 해 보자. 이때도 uniform분포를 이용해 0.0001에서 1사이의 값을 추출한다면 0.001에서 1 사이에 추출값의 90% 정도가 몰려 있게 될 것이다. 그런데 사실 $\alpha$값은 그 값이 작을수록 모형에 민감한 영향을 주기 때문에 그 값이 작을 수록 더 조밀하게 값들을 테스트 해야만 한다. 따라서 이런 하이퍼-파라미터는 값의 한쪽 극단으로 갈 수록 더 조밀한 샘플링이 필요하다. 이를 위해 **log scale 샘플링**을 이용한다.

$\alpha$값으로 0.0001에서 1 사이의 값을 테스트해야 한다면 우선 Uniform(-4, 1)에서 샘플링을 한 후(이 샘플링 값들을 $r$이라 하면), $10^r$을  $\alpha$값으로 사용한다. 이를 Python으로 구현한 코드는 아래와 같다.

```Python
r = -4 * np.random.rand()
alpha = np.power(10, r)
```

> 보다 일반적으로 $a < b$일 때 $10^a$와 $10^b$사이의 값을 $10^a$쪽에 조밀하게 추출해야 한다면 r = Uniform(a, b)에서 샘플링 한 후 $10^r$을 테스트 값으로 사용한다.

$\alpha$의 경우 작은 쪽 값을 더 조밀하게 샘플링 해야 했다. 그런데 $\beta$는 1보다 작은 값을 사용하며 1에 가까운 값일수록 더 민감하게 반응하므로 큰 값으로 갈 수록 더 조밀한 샘플링이 필요하다. 예를들어 0.9에서 0.999 사이의 $\beta$ 값을 테스트 해야 한다면 어떻게 해야 할까? 이런 경우 $\beta$를 샘플링하는 것이 아니라 우선 $1-\beta$를 log scale로 샘플링한다. 즉 $1-\beta$는 $10^{-1}$부터 $10^{-3}$까지의 범위를 갖으므로 우선 $r$을 Uniform(-3, -1)에서 샘플링하고, $1-\beta = 10^r$ 이고, $\beta$값으로 $1-10^r$을 사용한다.

$\beta$값이 작을 수록 모형에 민감하게 작용한다는 것을 다시 살펴보자. 앞서 지수 가중 평균은 $\beta$값에 따라 최근 $\frac{1}{1-\beta}$개 값들의 평균에 근사한다고 했었다. 예를들어 $\beta$가 0.9000 에서 0.9005의 범위일 경우 이는 최근 10개에서 10.05개 값의 평균을 내는 것이다. 반면 똑같이 0.0005 범위로서 $\beta$가 0.999에서 0.9995사이의 범위일 경우 이는 최근 1000개에서 2000개 값의 평균을 내는 것이다. 같은 길이의 $\beta$ 구간이지만 그 변화의 폭은 후자의 경우가 훨씬 크다.

### [1-3]. Hyperparameters tuning in practice: Pandas vs. Caviar

하이퍼-파라미터를 튜닝을 할 때 도움이 될만한 실질적인 팁 몇가지를 소개한다.

- **Cross-fertilization** : 자연어 처리, 컴퓨터 비전, 음성인식, 광고, 물류 등 다양한 분야에서 딥러닝이 사용되고 있는데, 어떤 분야의 하이퍼-파라미터 설정 기법이 다른 분야에 적용될 때도 있고 그렇지 않을 때도 있다. 이런 가능성 때문에 딥러닝을 적용한 다른 분야의 논문들을 연구하고 이런 기법들을 내 분야에 가져와 적용해보는 시도가 많다. 예를들어 컴퓨터 비전 분야에서 연구된 ConvNet이나 ResNet을 음성인식분야 효과적으로 적용된 것이나 음성인식분야의 기법들이 자연어처리에 적용된 사례가 있다.

- **Re-evaluate hyperparameter occasionally** : 지금 잘 맞는 하이퍼-파라미터를 찾았다 하더라도 알고리즘이 변경되거나 시간이 지나면서 데이터의 경향성이 달라지거나 서버를 업그레이드 했다는 등의 상황 변화에 맞는 더 나은 하이퍼-파라미터로 변경해야하는 상황이 오게 된다. 즉 주기적으로(몇개월만다?) 하이퍼-파라미터를 다시 평가하는 과정이 필요하다.

- **Pandas vs Caviar** : 다양한 후보의 하이퍼-파라미터 값들을 평가하는 방법은 크게 두가지 방식으로 나눌 수 있다. 
 - 첫번째는 다뤄야 하는 데이터의 양이 매우 크지만 시스템 성능이 좋지 않은 경우 (다양한 모형을 동시에 테스트 할 수 없으므로) 하이퍼-파라미터 값을 임의로 설정하고 한 모형을 학습시키면서 learning curve의 움직임에 따라 하이퍼-파라미터 값을 점차 변화시키는 방법을 사용한다. 이런 방식은 하나 혹은 단지 몇개의 모형을 점차 가다듬어가는 방식이므로 Babysitting방식 혹은 **Panda**(판다는 새끼 한 둘을 낳고 계속 키우므로) 방식이라 하자. 
  - 두번째 방식은 다뤄야 하는 데이터가 그리 크지 않고 컴퓨터 성능이 충분한 경우 다양한 하이퍼-파라미터가 다른 다양한 모형을 함께 학습시키면서 가장 좋은 learning curve를 보이는 하이퍼-파라미터 값을 선택하는 것이다. 이런 방식은 다양한 값들을 동시에 테스트하기 때문에 **Caviar**(케비어 등 물고기는 수많은 알을 낳고 이들을 동시에 키우므로) 방식이라고 부르기로 하자.

# [2]. Batch Normalization

### [2-1]. Normalizing activations in a network

최근 딥러닝과 관련된 아이디어 중 그 중요도가 높은 것중 하나가 '서게이 아이오피(Sergey loffe)'와 '크리스찬 제거디(Christian Szegedy)'가 2015년 발표한 논문인 '<u>Batch Normalizatoin : Accelerating Deep Network Training by Reducing Internal Covariate Shift</u>'이다.

이는 Batch normalization(=Batch norm)이란 방법인데, 이를 사용하면 신경망 모형이 매우 강건(robust)해지고, 하이퍼-파라미터를 튜닝도 더 쉬워진다. 또한 층이 깊은 신경망을 더 잘 학습시킬 수 있게 된다.

(로지스틱) 회귀 모형을 학습시킬 때 입력값들을 정규화(normalize)하면 학습 속도가 빨라진다는 것을 알고 있다. (역자주:엄밀히는 여기서 말하는 정규화(normalize는 표준화(standardize)를 의미한다.)

즉 입력값이 $X = (x_1, x_2, \cdots, x_p)$일 때 $X$대신 $\dot{X} = \frac{X - \mu}{\sigma^2}$를 사용하는 것인데, 이렇게 하면 찌그러진 cost contour가 동그랗게 바뀌기 때문에 파라미터들을 학습이 빠르고 정확해진다.

신경망에서는 각 레이어마다 파라미터들을 학습시켜야 하는데, 이때 $l$번째 레이어에 입력으로 들어오는 값은 $a^{[l-1]}$이고, 이 값들의 가중합니다.  $z^{[l]}$이다. (회귀모형과 마찬가지로) 이런 입력값을 표준화하면 각 레이어의 파라미터 $W^{[l]}$과 $b^{[l]}$를 학습시키는데 도움이 되지 않을까? 이 아이디어를 구체화 한 것이 바로 Batch normalization이다.

Batch normalization에서는 $a^{[l-1]}$ 혹은 $z^{[l]}$를 정규화 하는데, 둘 중 어느 것을 정규화 할지에 대해서는 의견이 분분하다. 이 강의에서는 $z^{[l]}$을 정규화할 것이며, 이 방법이 좀더 많이 사용된다.

(Implementing Batch Norm)

Batch normalization을 구현하는 방법에 대해 알아보자. $l$번째 레이어에서의 Batch norm을 생각해보면, $l-1$번째 레이어의 출력값 $a^{[l-1]}$($l=1$이라면 입력 값 $x$)가 $l$번째 레이어의 입력으로 들어와 가중합 $z^{[l]}$을 계산하고 이 값이 $l$번째 레이어의 activation function을 통과하면 $a^{[l]}$이 된다. 주의할 점은 Batch norm 값으로 $z^{[l]}$를 바로 사용하는 대신 이를 변환한 $\tilde{z}^{[l]}$을 사용한다는 것이다.

우선 $z_{\text{norm}}^{[l](i)}$는 아래와 같이 구하며,
$$
\begin{align}
\mu^{[l]} &= \frac{1}{m} \sum_i z^{[l](i)} \\
\sigma^{[l]} &= \frac{1}{m} \sum_i \left( z^{[l](i)} - \mu^{[l]} \right)^2 \\
z_{\text{norm}}^{[l](i)} &= \frac{z^{[l](i) - \mu^{[l]}}}{\sqrt{{\sigma^{[l]}}^2} + \epsilon^2}
\end{align}
$$
위 과정에서 분모에 $\epsilon$을 더해주는 것은 분모가 지나치게 작아지는 것을 막기 위함이다.(numerical stability). 

$z_{\text{norm}}^{[l](i)}$는 정규화를 통해 평균이 0이고 분산이 1인 분포로 변환되었는데 , 모든 레이어의 가중합이 동일한 분포를 갖는 것을 원하지 않기 때문에 여기에 아래와 같은 한 단계를 더 거쳐 $\tilde{z}^{[l]}$를 구하고, $z^{[l]}$대신 이 값을 activation function의 입력으로 사용한다.(이런 과정을 거치는 이유는 각 레이어의 activation function에 같은 분포값이 입력으로 들어오면 gradient또한 모두 같아지므로  학습이 정상적으로 진행되지 않기 때문이다.)

$$
\tilde{z}^{[l]} = \gamma z_{\text{norm}}^{[l](i)} + \beta$
$$

위 식에서 $\gamma$와 $\beta$는 다른 파라미터($w$, $b$)와 마찬가지 방식으로 모형 학습 과정에서 함께 학습시킬 수 있는 파라미터이다. 

사실 $\gamma$는 $z_{\text{norm}^{[l]}(i)}$의 분산에 대한 가중치이고, $\beta$는 $z_{\text{norm}^{[l]}(i)}$의 평균을 이동시키는 역할을 하는데, 예를들어 $\gamma = \sqrt{\sigma^2} + \epsilon^2$이고 $\beta = \mu$라면 이는 결국 정규화의 반대과정으로서 $\tilde{z^{[l]}(i)}$는 $z^{[l](i)}$와 같아지게 된다. 즉 $z^{[l](i)}$으로부터 $\tilde{z}^{[l](i)}$를 구하는 과정은 정규화 이후 평균을 이동시키고 분산을 스케일하는 것이다. $l$번째 레이어의 $n^l$개 $z^{ [l]}$에 모두 같은 $\gamma^{[l]}$와 $\beta^{[l]}$가 사용되므로 찌그러져 있는 cost contour가 원으로 변형되어 원점으로 이동한 후 다시 중심을 이동시키고 원의 너비를 변화시킨 것이라 생각할 수 있다.

### [2-2]. Fitting Batch Norm into a neural network

.

### [2-3]. Why does Batch Norm work?

.

### [2-4]. Batch Norm at test time

.

.

# [3]. Multi-class classification

### [3-1]. Training a softmax classifier

.

### [3-2]. Softmax regression

.

.

# [4]. Introduction to programming frameworks

### [4-1]. Deep Learning frameworks

.

### [4-2]. TensorFlow

.

.